# reaction-network (Demo Notebook): Enumerators

## Author: Matthew McDermott
## Last Updated: 11/16/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

In [2]:
import logging

from pymatgen.core.composition import Composition, Element
from pymatgen.ext.matproj import MPRester

from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator

from rxn_network.costs.softplus import Softplus
from rxn_network.entries.entry_set import GibbsEntrySet

In [37]:
with MPRester() as mpr:
    entries = mpr.get_entries_in_chemsys("Li-Co-O-Cl-Na", inc_structure="final")

  0%|          | 0/640 [00:00<?, ?it/s]

In [38]:
temp = 800
gibbs_entries = GibbsEntrySet.from_entries(entries, temp)

In [39]:
gibbs_entries.entries_list

[GibbsComputedEntry | mp-1008394 | Cl4 (Cl2)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-29537 | Cl8 O4 (Cl2O)
 Gibbs Energy (800 K) = 7.2843,
 BarinReferenceEntry | Cl2O
 Gibbs Energy (800 K) = 1.3854,
 GibbsComputedEntry | mp-31050 | Cl4 O14 (Cl2O7)
 Gibbs Energy (800 K) = 6.0153,
 GibbsComputedEntry | mp-1213922 | Cl8 O8 (ClO)
 Gibbs Energy (800 K) = 4.2758,
 GibbsComputedEntry | mp-1059242 | Cl1 O1 (ClO)
 Gibbs Energy (800 K) = 3.9316,
 BarinReferenceEntry | ClO
 Gibbs Energy (800 K) = 0.9436,
 GibbsComputedEntry | mp-23207 | Cl8 O16 (ClO2)
 Gibbs Energy (800 K) = 6.6715,
 GibbsComputedEntry | mp-1181699 | Cl2 O6 (ClO3)
 Gibbs Energy (800 K) = 5.6482,
 GibbsComputedEntry | mp-22869 | Cl4 O12 (ClO3)
 Gibbs Energy (800 K) = 4.8352,
 GibbsComputedEntry | mp-1181724 | Cl4 O20 (ClO5)
 Gibbs Energy (800 K) = 21.1363,
 GibbsComputedEntry | mp-1213825 | Cl4 O20 (ClO5)
 Gibbs Energy (800 K) = 19.3327,
 GibbsComputedEntry | mp-1181851 | Cl4 O24 (ClO6)
 Gibbs Energy (800 K) = 21.

In [40]:
filtered_entries = gibbs_entries.filter_by_stability(0.025)

In [41]:
filtered_entries.entries_list

[GibbsComputedEntry | mp-1008394 | Cl4 (Cl2)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-54 | Co2 (Co)
 Gibbs Energy (800 K) = 0.0000,
 BarinReferenceEntry | Co3O4
 Gibbs Energy (800 K) = -11.9841,
 BarinReferenceEntry | CoCl2
 Gibbs Energy (800 K) = -3.1838,
 GibbsComputedEntry | mp-1018134 | Li3 (Li)
 Gibbs Energy (800 K) = 0.0000,
 NISTReferenceEntry | Li2O
 Gibbs Energy (800 K) = -5.1219,
 BarinReferenceEntry | LiCl
 Gibbs Energy (800 K) = -6.1762,
 GibbsComputedEntry | mp-10172 | Na2 (Na)
 Gibbs Energy (800 K) = 0.0000,
 NISTReferenceEntry | Na2O
 Gibbs Energy (800 K) = -3.1935,
 BarinReferenceEntry | Na2O2
 Gibbs Energy (800 K) = -3.5140,
 NISTReferenceEntry | NaCl
 Gibbs Energy (800 K) = -3.4911,
 BarinReferenceEntry | NaClO4
 Gibbs Energy (800 K) = -5.7562,
 GibbsComputedEntry | mp-12957 | O8 (O2)
 Gibbs Energy (800 K) = 0.0000]

In [42]:
be = BasicEnumerator()

In [14]:
all_rxns = be.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
all_rxns

[YMnO3 + Mn2O3 -> 3 MnO2 + Y,
 0.5 Y2Mn2O7 + MnO -> YMnO3 + 0.5 Mn2O3,
 Mn5O8 + 2.861 Y -> 2.667 YMnO3 + 0.1944 YMn12,
 0.1667 Y2O3 + 0.5 Mn2O3 -> MnO2 + 0.3333 Y,
 8 MnO2 + 2 Y -> Y2Mn2O7 + 3 Mn2O3,
 Mn3O4 + 0.08333 Y -> 2 MnO2 + 0.08333 YMn12,
 MnO2 + 0.3077 Y -> 0.1538 Y2Mn2O7 + 0.2308 Mn3O4,
 YMnO3 + 1.667 MnO -> 0.06667 YMn12 + 0.9333 YMn2O5,
 Y2Mn2O7 + 22 MnO2 -> 2 YMn12 + 25.5 O2,
 2.2 MnO2 + 0.08 YMn12 -> Mn3O4 + 0.08 YMn2O5,
 Mn5O8 + Mn2O3 -> 2 O2 + 7 MnO,
 Mn + 1.5 MnO2 -> 0.5 Mn3O4 + MnO,
 27 MnO2 + 2 YMn12 -> Y2O3 + 51 MnO,
 YMnO3 + 0.875 Mn -> 0.375 Mn5O8 + Y,
 O2 + 2 MnO -> 2 MnO2,
 MnO2 + 2 MnO -> Mn3O4,
 Mn3O4 + Mn2O3 -> 2 MnO2 + 3 MnO,
 MnO2 + Mn2O3 -> 3 Mn + 2.5 O2,
 0.5 Mn3O4 + YMn2O5 -> YMnO3 + 0.5 Mn5O8,
 YMnO3 + MnO2 -> YMn2O5,
 MnO2 + Mn2O3 -> O2 + 3 MnO,
 YMnO3 + Mn2O3 -> MnO + YMn2O5,
 2 Y + 7 MnO -> Y2Mn2O7 + 5 Mn,
 0.08333 Mn + Mn2O3 -> 0.5833 Mn3O4 + 0.3333 MnO2,
 Mn + 0.3333 YMn2O5 -> 0.3333 Y + 1.667 MnO,
 51.5 MnO2 + YMn12 -> YMnO3 + 12.5 Mn5O8,
 Mn3O4 ->

In [45]:
be_target = BasicOpenEnumerator(["O2"],targets=["LiCoO2"])
ymno3_rxns = be_target.enumerate(filtered_entries)

BasicOpenEnumerator:   0%|          | 0/14 [00:00<?, ?it/s]

In [46]:
ymno3_rxns

[Co3O4 + 3 Li + O2 -> 3 LiCoO2,
 0.5 Li2O + 0.6667 Co3O4 -> LiCoO2 + Co + 0.5833 O2,
 Li2O + 2 Co + 1.5 O2 -> 2 LiCoO2,
 0.05584 Co3O4 + Li + 0.264 O2 -> 0.1675 LiCoO2 + 0.4162 Li2O,
 4 Li2O + Co -> LiCoO2 + 7 Li + O2,
 0.3571 Co3O4 + Li + 0.2857 O2 -> LiCoO2 + 0.07143 Co,
 Li2O + 0.3333 Co3O4 -> LiCoO2 + Li + 0.1667 O2,
 Li2O + 1.5 O2 + 2 CoCl2 -> 2 Cl2 + 2 LiCoO2,
 Co3O4 + 3 LiCl + O2 -> 1.5 Cl2 + 3 LiCoO2,
 Li2O + Co + 0.5 O2 -> LiCoO2 + Li,
 0.0203 Li2O + Co + 0.6701 O2 -> 0.04061 LiCoO2 + 0.3198 Co3O4,
 Li2O + 0.1667 O2 + 0.6667 CoCl2 -> 0.6667 LiCoO2 + 1.333 LiCl,
 0.6667 Co3O4 + Li -> LiCoO2 + Co + 0.3333 O2,
 Li2O + 0.6667 Co3O4 + 0.1667 O2 -> 2 LiCoO2]

In [20]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns = be_target.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
ymno3_rxns

[Y2O3 + Mn2O3 -> 2 YMnO3]